# Set 1
## Load data

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [ ]:
from tqdm import tqdm

import pickle
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

In [ ]:
top_performers = pd.DataFrame()

# Combine

In [ ]:
def cluster_plot_bokeh(grid):
    line_dash = ['solid', 'dashed', 'dotted', 'dotdash']
    color = ['blue', 'red', 'hotpink', 'green']
    p = figure(width=1400, height=500, x_axis_label='Number clusters', y_axis_label='Correlation silhouette score')
    p1 = figure(width=1400, height=500, x_axis_label='Number clusters', y_axis_label='Euclidian silhouette score')
    for i, alg in enumerate(grid):
        df = grid[alg].groupby(['n_clusters']).max().reset_index()
        df = df.sort_values('n_clusters')
        source = ColumnDataSource(df)
        p.line('n_clusters', 'silhouette_score_correlation', source=source,
                    legend_label=f'silhouette score {alg}', color=color[i])
        p1.line('n_clusters', 'silhouette_score_euclidean', source=source,
                    legend_label=f'silhouette score {alg}', color=color[i])

    p.add_layout(p.legend[0], 'right')
    p.legend.click_policy="hide"
    p1.add_layout(p1.legend[0], 'right')
    p1.legend.click_policy="hide"
    return column(p, p1)

In [ ]:
pd.options.display.width = 100
def get_individual_score(grid, i):
    top_df = []
    for alg in grid:
        print(f'_________________________________________{alg}_________________________________________')
        print()
        df = grid[alg].copy()
        if 'silhouette_score_manhattan' in df.columns:
            df = df.drop('silhouette_score_manhattan', axis=1)
        if 'calinski_harabasz_score' in df.columns:
            df = df.drop('calinski_harabasz_score', axis=1)
        if 'Fit_Time' in df.columns:
            df = df.drop('Fit_Time', axis=1)
        print(f'                        -----------------mean------------------')
        value1 = MinMaxScaler().fit_transform(df['silhouette_score_correlation'].values.reshape(-1, 1))
        value2 = MinMaxScaler().fit_transform(df['silhouette_score_euclidean'].values.reshape(-1, 1))
        df['mean'] = (value1 + value2) / 2
        #df = df.drop(['silhouette_score_correlation', 'silhouette_score_euclidean'], axis=1)
        df = df.sort_values(by='mean', ascending=False)
        df = df.drop('mean', axis=1)
        print(df.head(3))
        print()
        top_df.append([alg, df['silhouette_score_correlation'].values[0], df['silhouette_score_euclidean'].values[0]])
    with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet{i}/baseline_df.pkl', 'rb') as f:
        baseline = pickle.load(f)
    top_df.append(['Baseline', baseline['silhouette_score_correlation'].values[0], 
                   baseline['silhouette_score_euclidean'].values[0]])
    print('_________________________________________BEST_________________________________________')
    top_df = pd.DataFrame.from_records(top_df, columns=['Algorithm', 'silhouette_score_correlation', 
                                                      'silhouette_score_euclidean'])
    print(top_df)
    names = {
        1: 'CCLE',
        2: 'GPL570',
        4: 'TCGA'
    }
    top_df['Set'] = names[i]
    return top_df

In [ ]:
def get_results_plot(grid, i, end=15):
    plot_df = pd.DataFrame()
    for alg in grid:
        df = grid[alg].groupby(['n_clusters']).max().reset_index()
        df = df.sort_values('n_clusters')[['n_clusters','silhouette_score_euclidean', 'silhouette_score_correlation']]
        df['algorithm'] = alg
        plot_df = pd.concat([plot_df, df], ignore_index=True)
    plot_df = plot_df.dropna()
    # Add the baselilne
    with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet{i}/baseline_df.pkl', 'rb') as f:
        df = pickle.load(f)    
    plot_df = pd.concat([plot_df, df], ignore_index=True)
    fig, axs = plt.subplots(2,1,  figsize=(20,10), sharex=True)
    axs[0].set_xlim(2, end)
    axs[1].set_xlim(2, end)
    g = sns.lineplot(data=plot_df, x='n_clusters', y='silhouette_score_euclidean', hue='algorithm', ax=axs[0])
    g1 = sns.lineplot(data=plot_df, x='n_clusters', y='silhouette_score_correlation', hue='algorithm', ax=axs[1], legend=False)
    fig.suptitle(f'Set {i}', fontsize=24)
    plt.show()
    return plot_df

# Set 1

In [ ]:
i = 1
with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet{i}/Merged_results.pkl', 'rb') as f:
    results_grid = pickle.load(f)
end_df = get_results_plot(results_grid, i)

In [ ]:
top_df = get_individual_score(results_grid, i)

In [ ]:
top_performers = pd.concat([top_performers, top_df])
top_performers

# Set2

In [ ]:
i = 2
with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet{i}/Merged_results.pkl', 'rb') as f:
    results_grid = pickle.load(f)
end_df = get_results_plot(results_grid, i,15)

In [ ]:
top_df = get_individual_score(results_grid, i)

In [ ]:
with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet2/Merged_GridHDDC.pkl', 'rb') as f:
    grid = pickle.load(f)
grid[grid['K']==2]

In [ ]:
top_performers = pd.concat([top_performers, top_df])

# Set4

In [ ]:
i = 4
with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet{i}/Merged_results.pkl', 'rb') as f:
    results_grid = pickle.load(f)
end_df = get_results_plot(results_grid, i, 30)

In [ ]:
top_df = get_individual_score(results_grid, i)

In [ ]:
with open(f'/home/g0017139/UMCG_Thesis/Working_Code/Results/TSet4/Merged_GridHDDC.pkl', 'rb') as f:
    grid = pickle.load(f)
grid[grid['K']==19]

In [ ]:
top_performers = pd.concat([top_performers, top_df])

# Combine

In [ ]:
import seaborn as sns
sns.set(font_scale = 1.3,style="whitegrid")
fig, (ax1, ax2) = plt.subplots(1,2, figsize=(16,8))

sns.barplot(x="Set", y="silhouette_score_euclidean", hue="Algorithm", data=top_performers, ax=ax1)
sns.barplot(x="Set", y="silhouette_score_correlation", hue="Algorithm", data=top_performers, ax=ax2)

ax1.get_legend().remove()
ax1.set_ylabel('Euclidian distance')
ax2.set_ylabel('Correlation distance')
ax2.legend(bbox_to_anchor=(1.04,1), loc="upper left")
fig.suptitle('Best silhoutte scores')
plt.savefig(f'Results/Best_Scores.svg', dpi=1200)
